# **공연별 예매된 좌석, 예매되지 않은 좌석 테이블 구조 만들기**

In [1]:
import pandas as pd 
import sys
import numpy as np
import time

In [2]:
seat_information = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/seat_coordinate_info (symmetry point).csv",encoding = 'utf-8')
df = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/datadataframe_reduced_size.csv",encoding = 'utf-8')

In [3]:
seat_information.head(3)

,층,블록,열,넘버,전체_좌석,X,Y,Z,대칭점
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7


In [4]:
# merge 하기 위해 전체_좌석 컬럼을 seat 라는 컬럼 명으로 변경시커주자 

seat_information = seat_information.rename(columns = {'전체_좌석':'seat'})

display(seat_information.head())

,층,블록,열,넘버,seat,X,Y,Z,대칭점
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5


In [5]:
print('좌석 정보 데이터 프레임 : ', seat_information.shape)
print('메모리 정리 된 데이터 프레임 :', df.shape)

좌석 정보 데이터 프레임 :  (2505, 9)
메모리 정리 된 데이터 프레임 : (236966, 33)


# **해야 할 일**

각 공연 일, performance code 별 예매 된 좌석에 대해서는 예매 여부를 담은 테이블을 만들어야 함

In [6]:
print('유니크한 공연일 개수 : ',df['전체공연시간'].nunique())

유니크한 공연일 개수 :  162


# **빅 콘테스트 의문사항**

In [7]:
unique_combinations = df.groupby(['genre','performance_code', '전체공연시간']).size().reset_index(name='count')

cond = unique_combinations['count'] > 2505

unique_combinations[cond]

,genre,performance_code,전체공연시간,count
52,클래식,902,2019-03-14 20:00:00,2580
58,클래식,916,2019-05-19 17:00:00,2961
68,클래식,989,2023-05-21 17:00:00,2718
74,클래식,1182,2022-12-20 19:30:00,2558
109,클래식,1725,2021-03-31 19:30:00,3030
110,클래식,1828,2019-07-13 17:00:00,2723
111,클래식,1869,2019-07-02 20:00:00,2574
122,클래식,2158,2022-10-20 20:00:00,2743


전체 공연 좌석이 2505 개인데 2505개보다 많은 좌석이 판매된 경우가 있음

살펴보니 같은 좌석에서 다른 사람이 다른 시간에 예매를 한 것으로 나옴

이렇게 되면 해당 공연의 예매율은 100% 를 넘어갈텐데

우선 그룹핑 할때는 이런 일이 없도록 drop_duplicate 를 시행해주자

# **좌석 좌표와 병합 후 없는 좌석은 예매 여부 0으로 하기**

# **병합 전 seat_information.csv 파일 수정**

In [8]:
cond = seat_information['블록'].str.contains('BOX')

# Box석의 경우 BOX 의 번호가 열이 되도록 수정

seat_information.loc[cond, '열'] = seat_information.loc[cond, '블록'].map(lambda x: x[3:])
seat_information.loc[cond, '블록'] = seat_information.loc[cond, '블록'].map(lambda x: x[:3])

In [9]:
print('맨 위 5개의 행')
display(seat_information.head())

print('맨 아래 5개의 행')
display(seat_information.tail())

맨 위 5개의 행


,층,블록,열,넘버,seat,X,Y,Z,대칭점
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5


맨 아래 5개의 행


,층,블록,열,넘버,seat,X,Y,Z,대칭점
2500,2층,BOX,6,2,2층 BOX6 2,-1432,505,383,2층 BOX1 1
2501,2층,BOX,6,3,2층 BOX6 3,-1500,565,428,2층 BOX1 4
2502,2층,BOX,6,4,2층 BOX6 4,-1454,617,428,2층 BOX1 3
2503,2층,BOX,6,5,2층 BOX6 5,-1522,677,483,2층 BOX1 6
2504,2층,BOX,6,6,2층 BOX6 6,-1568,624,483,2층 BOX1 5


In [13]:
df.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'membership_type_6', 'tran_date', 'tran_time', 'play_date',
       'play_st_time', 'seat', 'price', 'ticket_cancel', 'discount_type',
       'performance_code', 'pre_open_date', 'open_date', 'genre', 'place',
       'running_time', 'intermission', 'member_yn', '공연연도', '공연월', '공연일',
       '공연연월', '전체공연시간', '전체거래시간', '층', '석', '세부좌석'],
      dtype='object')

In [14]:
class seat_merging:
    
    def __init__(self,data,seat):
        '''
        예매 된 공연 테이블과 좌석 정보를 담은 테이블을 가져오기
        '''
        self.df = data 
        self.seat = seat
        
        self.seat['예매여부'] = 1 # merging 후 특정 조건에 따라 예매 여부를 0으로 mapping 하여 수정 할 예정 
        
        
        self.genres_df_dict = {}
        self.genres_merged_dict = {}
        self.genres_num_count = {}

            
    def merging_seat_information(self):
        
        # 병합 후 채워야 하는 컬럼들 
        
        arr = ['play_date','play_st_time','ticket_cancel','performance_code',
                    'pre_open_date','open_date','genre','place','running_time','intermission',
                    '공연연도','공연월','공연일','공연연월','전체공연시간']
                
        unique_performance = sorted(self.df['전체공연시간'].unique())        

        merging_df_list = []
        
        for uni in unique_performance: # 장르 별, 공연 별로 merge 시키기
            
            cond = self.df['전체공연시간'] == uni
            
            cond_df = self.df[cond].copy()
            
            cancle_cond = self.df['ticket_cancel'] == 1 # 티켓을 취소한 경우도 제거해야 한다 
            
            cond_df = cond_df[~cancle_cond].copy()
            
            # 좌석의 중복값을 제거해야함 
            # 전체 거래시간 별로 오름차순 정렬 후 마지막에 중복값중 마지막 중복값 제외 drop 하기
            # 마지막 거래가 실제 거래라고 생각하기로 함
            
            cond_df = cond_df.sort_values(by = '전체거래시간')
            cond_df = cond_df.drop_duplicates(subset = 'seat', keep = 'last')
            
            # seat information 과 유니크한 공연 별 데이터 프레임을 병합 
            # 이렇게 하게 되면 merging_df 는 2505 행의 데이터프레임이 완성 될 것임 
            merging_df = pd.merge(self.seat[['seat','층', '블록', '열', '넘버', 'X', 'Y', 'Z', '대칭점', '무대_밑_좌우시야각',
                                            '무대_위_좌우시야각', '무대가_보이는_면적','예매여부']], cond_df, on = 'seat', how = 'left')
            
            
            not_purcahce_cond = merging_df['전체공연시간'].isna()
            
            merging_df.loc[not_purcahce_cond, '예매여부'] = 0 # 예매가 안됐던 경우에 대해서는 예매 여부를 0으로 변경하기 
            
            for fill_col in arr: 
                if merging_df[fill_col].notna().any():
                    original_value = merging_df.loc[merging_df[fill_col].notna(), fill_col].iloc[0]
                    merging_df[fill_col] = merging_df[fill_col].fillna(original_value)
                
            merging_df_list.append(merging_df) # merging_df_list 에 차곡 차곡 2505 석의 공연 별로 채워주자 
                    
        result = pd.concat(merging_df_list)
        
        return result

In [15]:
merging_method = seat_merging(df.drop(['층','석','세부좌석'],axis = 1),
                                seat_information) # 기존 df 의 층,석,세부좌석이 seat_information 과 같은 내용이니 하나는 drop 하자

In [16]:
merging_df = merging_method.merging_seat_information()

C:\Users\whileduck\AppData\Local\Temp\ipykernel_8816\1536485310.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cond_df = cond_df[~cancle_cond].copy()


KeyError: "['무대_밑_좌우시야각', '무대_위_좌우시야각', '무대가_보이는_면적'] not in index"

In [100]:
# 결과물 예시 

example = merged_dict['클래식']

print('예매여부가 1인 경우')
display(example[example['예매여부'] == 1].head(3))


print('예매여부가 0인 경우')
display(example[example['예매여부'] == 0].head(3))

예매여부가 1인 경우


,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,place,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간
0,1층 A블록1열 1,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9,21.802320,...,콘서트홀,120.0,15.0,N,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00
1,1층 A블록1열 2,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8,22.411843,...,콘서트홀,120.0,15.0,N,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00
2,1층 A블록1열 3,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7,23.021978,...,콘서트홀,120.0,15.0,N,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00


예매여부가 0인 경우


,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,place,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간
636,1층 D블록12열 6,1층,D블록,12,6,-810,1739,109,1층 B블록12열 7,21.672324,...,콘서트홀,120.0,15.0,NaN,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,NaN
661,1층 A블록13열 7,1층,A블록,13,7,1546,1697,130,1층 E블록13열 6,17.849806,...,콘서트홀,120.0,15.0,NaN,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,NaN
662,1층 A블록13열 8,1층,A블록,13,8,1497,1708,130,1층 E블록13열 5,18.106676,...,콘서트홀,120.0,15.0,NaN,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,NaN


In [104]:
genres = merging_method.genres


uniq_list = []
shapes = []
num_counts = []
for g in genres: # 잘 체크 되었는지 확인 
    
    genre_df = merged_dict[g]

    grouping_df = genre_df.groupby('전체공연시간').size().reset_index(name = 'count')
    
    unqiue_value = grouping_df['count'].unique()    
    
    uniq_list.append(unqiue_value[0])
    shapes.append(genre_df.shape)
    
result = pd.DataFrame({
    '장르':genres,
    'shape' : shapes,
    '장르 별 공연 횟수': merging_method.genres_num_count.values(),
    '각 공연 별 유니크한 좌석 수' : uniq_list
})

display(result)

,장르,shape,장르 별 공연 횟수,각 공연 별 유니크한 좌석 수
0,독주,"(105210, 42)",42,2505
1,교향곡,"(906810, 42)",362,2505
2,클래식,"(405810, 42)",162,2505
3,오페라,"(35070, 42)",14,2505
4,합창,"(175350, 42)",70,2505
5,성악,"(77655, 42)",31,2505
6,실내악,"(65130, 42)",26,2505
7,콘서트,"(45090, 42)",18,2505
8,복합장르,"(15030, 42)",6,2505
9,기타,"(7515, 42)",3,2505


# **잘 완료 되었고 깃허브에서 연동 할 수 있도록 각 데이터 프레임의 용량을 낮춘 후 저장하자**

In [105]:
class DataExploratioin:
    '''
    데이터 탐색 시 사용 가능한 Class 

    기존 존재하는 프레임워크들을 이용하여 자주 이용하는 프레임워크들을 활용하여 나만의 분석 툴을 만들려고 함 

    데이터 요약, 결측값 처리 등의 내용이 담겨있는 class 
    '''

    def __init__(self, data):
        self.data = data

    def summarize(self):
        '''
        데이터를 초창기에 요약해주는 method
        '''

        cols = self.data.columns

        size = round(sys.getsizeof(self.data) / 1024 ** 2, 2)

        print(f'data size : {size}MB')

        self.result = pd.DataFrame()

        self.result['Dtype'] = self.data.dtypes.values
        self.result['Count'] = self.data.count().values
        self.result['Nunique'] = self.data.nunique().values
        self.result['Missing value'] = self.data.isna().sum().values
        self.result['Missing %'] = [str(round(
            missing / len(self.data), 2) * 100) + '%' for missing in self.result['Missing value']]
        self.result['Most Freq Value'] = self.data.mode().iloc[0].values

        freq_prop = []

        for i, col in enumerate(cols):

            raw_data = self.data.loc[~self.data[col].isna(), col]
            freq_value = self.result['Most Freq Value'].iloc[i]

            prop = np.mean(
                np.array(raw_data == freq_value)
            )

            prop_str = str(round(np.mean(prop) * 100, 1)) + '%'

            if prop_str == 'nan%':
                freq_prop.append(self.result['Missing %'].iloc[i])
            else:
                freq_prop.append(prop_str)

        self.result['Most Freq Value %'] = freq_prop

        self.result['Min'] = self.data.describe(include='all').T['min'].values
        self.result['Max'] = self.data.describe(include='all').T['max'].values
        self.result['Mean'] = self.data.describe(
            include='all').T['mean'].values
        self.result['Median'] = self.data.describe(
            include='all').T['50%'].values
        
        memory = (self.data.memory_usage(deep = True) // 1024 **2).values[1:] # index 의 usage 는 제외하고 보자 

        
        self.result['MB'] = [str(m) + ' mb' for m in memory]
        self.result = self.result.set_index(cols)

        self.result = self.result.fillna('-')

        display(self.result)
    
    
    def progress_bar(self,iterable, total_blocks = 10):
        
        total_items = len(iterable)
        block_size = total_items // total_blocks
        
        for i, item in enumerate(iterable, start=1):
            if i % block_size == 0 or i == total_items:
                progress = (i / total_items) * 100
                blocks = int(progress / (100 / total_blocks))
                empty_blocks = total_blocks - blocks
                progress_bar = '■' * blocks + '▢' * empty_blocks
                print(f"\rProgress: [{progress_bar}] {progress:.2f}%", end='', flush=True)
            yield item
            time.sleep(0.0000001)
    
    def reduce_size(self,iwantdisplay = False):
                
        original_size = round(sys.getsizeof(self.data) / 1024 ** 2,2)
        
        df = self.data.copy()
        
        for col in self.progress_bar(df.columns):
            
            dtp = df[col].dtype
            
            if dtp == 'object':
                df[col] = df[col].astype('category')
            else: # numeric type이면 
                
                if min(df[col]) >= 0 : # 부호가 없다면 unit 으로 변경해줘도 된다.
                    max_value = max(df[col])
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits: # 최소한의 비트로 표현 될 수 있게 dtype 변경 
                        if max_value < 2 ** bit:
                            # 결측치가 있는 경우 astype 으로 변경하지 못하니 결측치를 채워준 후 변경하고 다시 결측치를 채우자 
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'uint{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
                else: # 부호가 있다면 int type 으로 바꿔주자 
                    
                    max_value = max(abs(min(df[col])), max(df[col]))
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits:
                        if max_value < 2 ** bit:
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'int{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
        print('\n')
                        
        after_size = round(sys.getsizeof(df) / 1024 ** 2,2)
        
        # 바꾼 후 결과 보여주기 
        
        if iwantdisplay:    
            after = DataExploratioin(df)
            after.summarize()
            
        print(f'\n {original_size}MB -> {after_size}MB')
            
        return df

In [106]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data'

for genre, data in merged_dict.items():
    
    ep = DataExploratioin(data)
    
    reduce_data = ep.reduce_size()
    
    reduce_data.to_csv(f'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/{genre}_빈좌석포함데이터.csv', index = False)

Progress: [■■■■■■■■■■] 100.00%


 168.77MB -> 10.9MB
Progress: [■■■■■■■■■■] 100.00%


 1507.37MB -> 87.14MB
Progress: [■■■■■■■■■■] 100.00%


 674.54MB -> 39.11MB
Progress: [■■■■■■■■■■] 100.00%


 58.13MB -> 3.8MB
Progress: [■■■■■■■■■■] 100.00%


 294.68MB -> 15.74MB
Progress: [■■■■■■■■■■] 100.00%


 128.51MB -> 7.43MB
Progress: [■■■■■■■■■■] 100.00%


 107.56MB -> 6.55MB
Progress: [■■■■■■■■■■] 100.00%


 74.53MB -> 4.59MB
Progress: [■■■■■■■■■■] 100.00%


 25.01MB -> 2.08MB
Progress: [■■■■■■■■■■] 100.00%


 12.64MB -> 1.33MB
Progress: [■■■■■■■■■■] 100.00%


 7.97MB -> 1.12MB
Progress: [■■■■■■■■■■] 100.00%


 4.17MB -> 0.97MB
Progress: [■■■■■■■■■■] 100.00%


 3.85MB -> 0.91MB


In [108]:
seat_information.to_csv('seat_information_with_area_ver2.csv',index = False) # 수정된 seat_information 저장